## combine all country road networks and make into single file 

In [2]:
import glob
import geopandas as gp

folder_list=['BDI','DJI','ERI','ETH','KEN','RWA','SDN','SOM','SSD','TZA','UGA']


folderpath='/home/data_folder/osm_data/'

db_cont=[]
for folderl in folder_list:
    osm_road_shp=f'{folderpath}{folderl}/gis_osm_roads_free_1.shp'
    db=gp.read_file(osm_road_shp)
    db_cont.append(db)
    db=[]

In [4]:
import pandas as pd
db=pd.concat(db_cont)

In [5]:
db.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 4351142 entries, 0 to 545710
Data columns (total 11 columns):
 #   Column    Dtype   
---  ------    -----   
 0   osm_id    object  
 1   code      int64   
 2   fclass    object  
 3   name      object  
 4   ref       object  
 5   oneway    object  
 6   maxspeed  int64   
 7   layer     int64   
 8   bridge    object  
 9   tunnel    object  
 10  geometry  geometry
dtypes: geometry(1), int64(3), object(7)
memory usage: 398.4+ MB


In [6]:
db.to_file('/home/data_folder/osm_data/ea_road_networks.shp')
#took neary 30+30 minutes to finsih this step 

## make 5x5 grids of 1km resolution 

In [7]:
import glob
import rasterio as rio
import geopandas as gp
import pandas as pd
from shapely.geometry import box
import numpy as np
import ntpath

tiffiles=glob.glob('/home/data_folder/ea_dem/*.tif')

int_tiff=[]
dem_fl_name=[]
for tifl in tiffiles:
    ra = rio.open(tifl)
    bounds  = ra.bounds
    tiff_geom=box(*bounds)
    int_tiff.append(tiff_geom)
    dem_name=ntpath.basename(tifl).split('.')[0]
    dem_fl_name.append(dem_name)

db=pd.DataFrame()
db['geometry']=int_tiff

db['sno']=np.arange(1,len(int_tiff)+1)
db['dem_name']=dem_fl_name

db1=gp.GeoDataFrame(db)
db1.to_file('/home/data_folder/ea_merit_dem_5deg_grid.shp')

19.999583333333334 -0.0004166666666662877 24.999583333333334 4.999583333333334
500 500


In [9]:
import argparse
import pandas as pd
from shapely.geometry import Polygon, mapping
#import simplekml
import numpy as np
import itertools
import geopandas as gp
import ntpath
import math

import shapely.wkt


def gridcreator(swlong,swlat,gridx,gridy,nx,ny):
    """
    creates polygon grids as a geometry list for the speciifed variables

    Parameters
    ----------
    swlong : south west longitude value in degree decimals (float)
    swlat : south west latitude value in degree decimals (float)
    gridx : width size of the grid required in degress  (float)
    gridy : width size of the grid required in degress  (float)
    nx : number of grids required in x direction (integer)
    ny : number of grids required in y direction (integer)
            
    Returns
    -------
    a list of polygons geometry for the given variables
  
    Notes
    -------
    module for grid computation, effective to southern parts, 
    it really shrinking the grid size while moving to north
    """
    nelong=swlong+(gridx*nx)
    nelat=swlat+(gridy*ny)
    xmin,ymin,xmax,ymax= swlong, swlat, nelong, nelat
    gridWidth = gridx
    gridHeight = gridy
    rows = round((ymax-ymin)/gridHeight)
    cols = round((xmax-xmin)/gridWidth)
    ringXleftOrigin = xmin
    ringXrightOrigin = xmin + gridWidth
    ringYtopOrigin = ymax
    ringYbottomOrigin = ymax-gridHeight
    polygun1=[]
    for i in np.arange(cols):
        ringYtop = ringYtopOrigin
        ringYbottom =ringYbottomOrigin
        for j in np.arange(rows):
            polygon = Polygon([(ringXleftOrigin, ringYtop), (ringXrightOrigin, ringYtop), (ringXrightOrigin, ringYbottom), (ringXleftOrigin, ringYbottom)])
            polygun1.append(polygon)
            ringYtop = ringYtop - gridHeight
            ringYbottom = ringYbottom - gridHeight
        ringXleftOrigin = ringXleftOrigin + gridWidth
        ringXrightOrigin = ringXrightOrigin + gridWidth
    return polygun1   

def maille_counter(grid_df):
    """
    creates a pandas datframe with Maille and Maille_X and Maille_Y columns
    Parameters
    ----------
    grid_df : a pandas dataframe with polygon geometries
            
    Returns
    -------
    a pandas dataframe with Maille columns
    """
    grid_df.columns=['geometry']
    grid_df['gn']=np.arange(1,len(grid_df.index)+1,1)
    wef=grid_df['gn'].to_numpy()
    # it's a gymnastics to get sw count for grids
    o=wef.reshape(nx, ny)
    p=o.T
    q=np.ravel(p,order='F')
    b=wef.reshape(ny, nx)
    c=b.T
    d=np.rot90(c, 1)
    e=np.ravel(d,order='F')
    qq=pd.DataFrame()
    qq['gn']=q
    qq['Maille']=e
    qq1=qq.sort_values('Maille')
    ww1=pd.merge(grid_df,qq1)
    #this is for giving the column(Maille_X) and row(Maille_Y) numbering
    rows=np.arange(1,ny+1,1)
    cols=np.arange(1,nx+1,1)
    rows1=[rows] * nx
    rows2=[item for sublist in rows1 for item in sublist]
    rows3=np.fliplr([rows2])[0]
    cols1=list(itertools.chain.from_iterable(itertools.repeat(x,ny) for x in cols))
    ww1['Maille_Y']=rows3
    ww1['Maille_X']=cols1
    return ww1



def xy_maker(maille_grid,epsg_code):
    """
    creates a geopandas dataframe with centroid, lower end and upper end cordinate details
    calculates grid area

    Parameters
    ----------
    maille_grid : Dataframe with Maille grids
            
    Returns
    -------
    Geopandas dataframe
    """
    maille_grid['centroid']=maille_grid['geometry'].map(lambda x: x.centroid)
    maille_grid["X"] = maille_grid.centroid.map(lambda p: p.x)
    maille_grid["Y"] = maille_grid.centroid.map(lambda p: p.y)
    maille_grid['X1']=maille_grid['geometry'].map(lambda x: x.exterior.coords[3][0])
    maille_grid['Y1']=maille_grid['geometry'].map(lambda x: x.exterior.coords[3][1])
    maille_grid['X2']=maille_grid['geometry'].map(lambda x: x.exterior.coords[1][0])
    maille_grid['Y2']=maille_grid['geometry'].map(lambda x: x.exterior.coords[1][1])
    maille_grid.drop(['centroid','gn'], axis=1, inplace=True)
    maille_grid1=maille_grid.sort_values('Maille')
    maille_grid2=gp.GeoDataFrame(maille_grid1)
    maille_grid2.crs = {'init' :'epsg:4326'}
    maille_grid2['area_cell']=maille_grid2['geometry'].to_crs({'init': 'epsg:{0}'.format(epsg_code)})\
               .map(lambda p: p.area / 10**6)
    #maille_grid2['area_cell']=maille_grid2['geometry'].map(lambda p: p.area  / 1e+6 )
    maille_grid3=maille_grid2.reset_index()
    maille_grid3.drop(['index'], axis=1, inplace=True)
    return maille_grid3

def path_leaf(path):
    """
    Get the name of a file without any extension from given path

    Parameters
    ----------
    path : file full path with extension
    
    Returns
    -------
    str
       filename in the path without extension

    """
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)


In [31]:
swlong,swlat,nelong,nelat=db1['geometry'][37].bounds
print(swlong,swlat,nelong,nelat)
#gridx,gridy,nx,ny
gridx=0.01
gridy=0.01
nx=round((nelong-swlong)/gridx)
ny=round((nelat-swlat)/gridy)
print(nx,ny)
grids_poly=gridcreator(swlong,swlat,gridx,gridy,nx,ny)
grid_df=pd.DataFrame(grids_poly)
maille_grid_df=maille_counter(grid_df)

#epsg_code=crs_find_centroid(swlong,swlat,gridx,gridy,nx,ny)
#ue_grid=xy_maker(maille_grid_df,epsg_code)

34.999583333333334 -5.000416666666666 39.999583333333334 -0.00041666666666590873
500 500


In [32]:
maille_grid_df1=gp.GeoDataFrame(maille_grid_df)
maille_grid_df1.to_file('/home/data_folder/osm_data/s05e035_dem.shp')

In [29]:
#db1[['dem_name','sno']][37]

db1.iloc[[37],:]

,geometry,sno,dem_name
37,"POLYGON ((39.99958 -5.00042, 39.99958 -0.00042...",38,s05e035_dem


In [ ]:
## subset grids

In [2]:
import fiona
from shapely.geometry import shape, mapping
import rtree
import glob
import pandas as pd
import os
import ntpath
import time
start_time = time.time()


gridlist=['/home/data_folder/osm_data/s05e035_dem.shp']


def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)



def intersect(road_shp_layer,boundary_shp,grid_name,output_shp):
    with fiona.open(boundary_shp, 'r') as boundary_shp_layer:
        # We copy schema and add the  new property for the new resulting shp
        schema = road_shp_layer.schema.copy()
        # We open a first empty shp to write new content from both others shp
        with fiona.open(output_shp, 'w', 'ESRI Shapefile', schema) as output_shp_layer:
            index = rtree.index.Index()
            for feat1 in boundary_shp_layer:
                fid = int(feat1['id'])
                geom1 = shape(feat1['geometry'])
                index.insert(fid, geom1.bounds)
            for feat2 in road_shp_layer:
                geom2 = shape(feat2['geometry'])
                for fid in list(index.intersection(geom2.bounds)):
                    if fid != int(feat2['id']):
                        feat1 = boundary_shp_layer[fid]
                        geom1 = shape(feat1['geometry'])
                        if geom1.intersects(geom2):
                            # We take attributes from ctSHP
                            props = feat2['properties']
                            # Then append the uid attribute we want from the other shp
                            props['code'] = grid_name
                            geom3=geom1.intersection(geom2)
                            #props['length']=geom3.length*100
                            # Add the content to the right schema in the new shp
                            output_shp_layer.write({
                                'properties': props,
                                'geometry': mapping(geom1.intersection(geom2))
                            })

                                
road_shp  = '/home/data_folder/osm_data/ea_road_networks.shp'
road_shp_layer= fiona.open(road_shp, 'r')


for grid_path in gridlist:
    #shpfiles1=(os.path.splitext(city)[0])
    boundary_shp=grid_path
    grid_name=ntpath.basename(grid_path)
    output_shp=f'{os.path.dirname(grid_path)}/{grid_name}_road.shp'
    intersect(road_shp_layer,boundary_shp,grid_name,output_shp)
    print("completed "+grid_name)
    
print("--- %s seconds ---" % (time.time() - start_time))

NameError: name 'shpfiles2' is not defined

## grid level intersection 
1. This is better to avoid the intermediate step

### creating the grid file 

In [12]:
import geopandas as gp

db1=gp.read_file('/home/data_folder/osm_data/ea_5x5_grid.shp')



for idx, row in db1.iterrows():
    swlong,swlat,nelong,nelat=row['geometry'].bounds
    print(swlong,swlat,nelong,nelat)
    #gridx,gridy,nx,ny
    gridx=0.01
    gridy=0.01
    nx=round((nelong-swlong)/gridx)
    ny=round((nelat-swlat)/gridy)
    print(nx,ny)
    grids_poly=gridcreator(swlong,swlat,gridx,gridy,nx,ny)
    grid_df=pd.DataFrame(grids_poly)
    maille_grid_df=maille_counter(grid_df)
    maille_grid_df1=gp.GeoDataFrame(maille_grid_df)
    output_name=row['dem_name']
    maille_grid_df1.to_file(f'/home/data_folder/osm_data/{output_name}.shp')
    maille_grid_df=[]
    maille_grid_df1=[]
    grids_poly=[]
    grid_df=[]

24.999583333333334 -0.0004166666666662877 29.999583333333334 4.999583333333334
500 500
29.999583333333334 -0.0004166666666662877 34.999583333333334 4.999583333333334
500 500
34.999583333333334 -0.0004166666666662877 39.999583333333334 4.999583333333334
500 500
39.999583333333334 -0.0004166666666662877 44.999583333333334 4.999583333333334
500 500
44.999583333333334 -0.0004166666666662877 49.999583333333334 4.999583333333334
500 500
19.999583333333334 4.999583333333334 24.999583333333334 9.999583333333334
500 500
24.999583333333334 4.999583333333334 29.999583333333334 9.999583333333334
500 500
29.999583333333334 4.999583333333334 34.999583333333334 9.999583333333334
500 500
34.999583333333334 4.999583333333334 39.999583333333334 9.999583333333334
500 500
39.999583333333334 4.999583333333334 44.999583333333334 9.999583333333334
500 500
44.999583333333334 4.999583333333334 49.999583333333334 9.999583333333334
500 500
49.999583333333334 4.999583333333334 54.999583333333334 9.999583333333334

In [11]:
db1

,sno,dem_name,geometry
0,2,n00e025_dem,"POLYGON ((29.99958 -0.00042, 24.99958 -0.00042..."
1,3,n00e030_dem,"POLYGON ((34.99958 -0.00042, 29.99958 -0.00042..."
2,4,n00e035_dem,"POLYGON ((39.99958 -0.00042, 34.99958 -0.00042..."
3,5,n00e040_dem,"POLYGON ((44.99958 -0.00042, 39.99958 -0.00042..."
4,6,n00e045_dem,"POLYGON ((49.99958 -0.00042, 44.99958 -0.00042..."
5,7,n05e020_dem,"POLYGON ((24.99958 4.99958, 19.99958 4.99958, ..."
6,8,n05e025_dem,"POLYGON ((29.99958 4.99958, 24.99958 4.99958, ..."
7,9,n05e030_dem,"POLYGON ((34.99958 4.99958, 29.99958 4.99958, ..."
8,10,n05e035_dem,"POLYGON ((39.99958 4.99958, 34.99958 4.99958, ..."
9,11,n05e040_dem,"POLYGON ((44.99958 4.99958, 39.99958 4.99958, ..."


In [7]:
import fiona
from shapely.geometry import shape, mapping
import rtree
import glob
import pandas as pd
import os
import ntpath
import time
start_time = time.time()

def intersect(road_shp_layer,boundary_shp,grid_name,output_shp):
    with fiona.open(boundary_shp, 'r') as boundary_shp_layer:
        # We copy schema and add the  new property for the new resulting shp
        schema = road_shp_layer.schema.copy()
        schema['properties']['gno'] = 'int:10'
        schema['properties']['length'] = 'int:10'
        # We open a first empty shp to write new content from both others shp
        with fiona.open(output_shp, 'w', 'ESRI Shapefile', schema) as output_shp_layer:
            index = rtree.index.Index()
            for feat1 in boundary_shp_layer:
                fid = int(feat1['id'])
                geom1 = shape(feat1['geometry'])
                index.insert(fid, geom1.bounds)
            for feat2 in road_shp_layer:
                geom2 = shape(feat2['geometry'])
                for fid in list(index.intersection(geom2.bounds)):
                    if fid != int(feat2['id']):
                        feat1 = boundary_shp_layer[fid]
                        geom1 = shape(feat1['geometry'])
                        if geom1.intersects(geom2):
                            # We take attributes from ctSHP
                            props = feat2['properties']
                            # Then append the uid attribute we want from the other shp
                            props['code'] = grid_name
                            geom3=geom1.intersection(geom2)
                            if geom3.geom_type=='GeometryCollection':
                                print("empty geometry")
                            elif geom3.geom_type=='Point':
                                print(props)
                            else:
                                props['gno'] = feat1['properties']['Maille']
                                props['length']=geom3.length*100
                                output_shp_layer.write({
                                  'properties': props,
                                  'geometry': mapping(geom1.intersection(geom2))
                                })

                                
road_shp  = '/home/data_folder/osm_data/ea_road_networks.shp'
road_shp_layer= fiona.open(road_shp, 'r')

gridlist=['/home/data_folder/osm_data/s05e035_dem.shp']

for grid_path in gridlist:
    #shpfiles1=(os.path.splitext(city)[0])
    boundary_shp=grid_path
    grid_name=ntpath.basename(grid_path)
    output_shp=f'{os.path.dirname(grid_path)}/{grid_name}_road_grid_stat.shp'
    #intersect(road_shp_layer,boundary_shp,grid_name,output_shp)
    print("completed "+grid_name)
    
print("--- %s seconds ---" % (time.time() - start_time))

completed s05e035_dem.shp
--- 1676.3988709449768 seconds ---


In [13]:
db1=gp.read_file('/home/data_folder/osm_data/ea_5x5_grid.shp')

road_shp  = '/home/data_folder/osm_data/ea_road_networks.shp'
road_shp_layer= fiona.open(road_shp, 'r')
start_time = time.time()
for idx, row in db1.iterrows():
    grid_name=row['dem_name']
    boundary_shp=f'/home/data_folder/osm_data/{grid_name}.shp'
    output_shp=f'/home/data_folder/osm_data/{grid_name}_road_grid_stat.shp'
    ###intersect(road_shp_layer,boundary_shp,grid_name,output_shp)
    print(f'completed {grid_name}')
    
print("--- %s seconds ---" % (time.time() - start_time))
#took --- 34199.34253573418 seconds ---

completed n00e025_dem
completed n00e030_dem
completed n00e035_dem
completed n00e040_dem
completed n00e045_dem
completed n05e020_dem
completed n05e025_dem
completed n05e030_dem
completed n05e035_dem
completed n05e040_dem
completed n05e045_dem
completed n05e050_dem
completed n10e020_dem
completed n10e025_dem
completed n10e030_dem
completed n10e035_dem
completed n10e040_dem
completed n10e045_dem
completed n10e050_dem
completed n15e020_dem
completed n15e025_dem
completed n15e030_dem
completed n15e035_dem
completed n20e025_dem
completed n20e030_dem
completed n20e035_dem
completed s05e025_dem
completed s05e030_dem
completed s05e035_dem
completed s05e040_dem
completed s10e025_dem
completed s10e030_dem
completed s10e035_dem
completed s15e030_dem
completed s15e035_dem
completed s15e040_dem
--- 34199.34253573418 seconds ---


In [5]:
os.path.dirname(gridlist[0])

'/home/data_folder/osm_data'

In [5]:
import geopandas as gp

grid_name='n00e025_dem'
db=gp.read_file(f'/home/data_folder/osm_data/ea_road_networks.shp')
db1=db.drop_duplicates('fclass')
db1


KeyboardInterrupt



In [1]:
import geopandas as gp
import time
start_time = time.time()

grid_name='n00e025_dem'
db=gp.read_file(f'/home/data_folder/osm_data/{grid_name}_road_grid_stat.shp')
db1=db.drop_duplicates('fclass')

print("--- %s seconds ---" % (time.time() - start_time))

--- 1.9922327995300293 seconds ---


In [4]:
import numpy as np
import geopandas as gp
import pandas as pd




rd0=gp.read_file(f'/home/data_folder/osm_data/{grid_name}_road_grid_stat.shp')
rd0.crs = {'init': 'epsg:4326', 'no_defs': True}
rd= rd0.to_crs({'init': 'epsg:32644'})
rd['length1']=rd['geometry'].length/1000
rd

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,gno,length,geometry,length1
0,20420320,0,primary,Faraksika-Maridi Road,14,B,0,0,F,F,249467,1,"LINESTRING (-6141019.891 882625.642, -6140946....",1.917762
1,20420320,0,primary,Faraksika-Maridi Road,14,B,0,0,F,F,249967,0,"LINESTRING (-6139550.026 883820.534, -6139451....",0.655237
2,20420320,0,primary,Faraksika-Maridi Road,14,B,0,0,F,F,249968,0,"LINESTRING (-6139057.701 884252.613, -6138929....",0.374071
3,20420320,0,primary,Faraksika-Maridi Road,14,B,0,0,F,F,248463,0,"LINESTRING (-6148370.392 880690.450, -6148283....",1.605318
4,20420320,0,primary,Faraksika-Maridi Road,14,B,0,0,F,F,248963,0,"LINESTRING (-6146807.869 881058.363, -6146646....",0.277972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29492,957102603,0,unclassified,None,None,B,0,0,F,F,5476,0,"LINESTRING (-6172077.098 19327.498, -6172042.5...",0.216379
29493,958884997,0,path,None,None,B,0,0,F,F,7997,0,"LINESTRING (-6134380.944 26824.908, -6134395.5...",0.063222
29494,967222110,0,footway,None,None,B,0,0,F,F,1976,0,"LINESTRING (-6171651.969 5972.215, -6171636.17...",0.211849
29495,1040800138,0,path,None,None,B,0,0,F,F,30997,0,"LINESTRING (-6134054.491 108624.289, -6134054....",0.149791


## categories the road netwrok and generate density maps

## to get the epsg code for each grids

In [2]:
import geopandas as gp


grid_db=gp.read_file('/home/data_folder/osm_data/ea_5x5_grid.shp')

#grid_db = grid_db.set_crs('epsg:4326')

#db1['epsg']=db1['geometry'].apply(lambda x: x.estimate_utm_crs().to_epsg())

#germany.estimate_utm_crs(


#world = gp.read_file(gp.datasets.get_path("naturalearth_lowres"))

#germany = world.loc[world.name == "Germany"]

#germany.estimate_utm_crs().to_epsg()
#germany

get_epsg_cont=[]
for idx,row in grid_db.iterrows():
    grid_db_sr=grid_db.loc[[idx]]
    grid_db_sr = grid_db_sr.set_crs('epsg:4326')
    get_epsg=grid_db_sr.estimate_utm_crs().to_epsg()
    get_epsg_cont.append(get_epsg)
    
grid_db['epsg']=get_epsg_cont
#grid_db1=grid_db.drop_duplicates('epsg')
#grid_db1

## to get road density in particular grids in csv format
1. for loop on every shape file of 5x5 grid
2. calculate the distance using epsg
3. replace the fclass column with the dctinary and make new_fclass
4. save the csv file with grid number, name of 5x5 grid, distance, new_fclass
5. do a group by pivot the table where the new_fclass becomes columns
6. attach the pivoted datframe to the 5x5 grid and fill the empty grid values for new_fclass
7. make the grid centroid and make into lat and long of datframe
8. save the dataframe as csv
9. plot the centroid lat long of all grid for each new_fclass classes



In [13]:
import pandas as pd
import time
import numpy as np

rdc=pd.read_csv('/home/ea-ibf-climada/exposure/roads/road_dict.csv')
bus=rdc.set_index('type')['cat2'].to_dict()
bus['nan'] = 'unclassified'

start_time = time.time()

for idx, row in grid_db.iterrows():
    grid_name=row['dem_name']
    rd0=gp.read_file(f'/home/data_folder/osm_data/{grid_name}_road_grid_stat.shp')
    rd0.crs = {'init': 'epsg:4326', 'no_defs': True}
    rd_grid_epsg=row['epsg']
    rd= rd0.to_crs({'init': f'epsg:{rd_grid_epsg}'})
    rd['fclass_length_km']=rd['geometry'].length/1000
    stli=rd.fclass.tolist()
    stli2=[]
    for lt in stli:
        L2 = str(lt)
        L3=[L2]
        stli2.append(L3)
    a=[]
    for lt in stli2:
        L2 = [bus[x] for x in lt]
        a.append(L2)
    ab=np.array(a)
    rd['road_category']=ab
    rd['grid_name']=grid_name
    rd1=rd[['gno','fclass_length_km','road_category','grid_name']]
    rd1.to_csv(f'/home/data_folder/osm_data/road_gno_cat_raw/{grid_name}.csv',index=False)
    print("--- %s seconds ---" % (time.time() - start_time))

/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


--- 3.2754392623901367 seconds ---


/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


--- 142.02620458602905 seconds ---


/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


...

--- 1232.2402493953705 seconds ---


/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


--- 1242.8299915790558 seconds ---


/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/root/miniconda3/envs/climada_env/lib/python3.8/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


--- 1245.16108584404 seconds ---


### group pivoting

In [34]:
import pandas as pd

start_time = time.time()
grid_db=gp.read_file('/home/data_folder/osm_data/ea_5x5_grid.shp')

for idx,row in grid_db.iterrows():
    grid_name=row['dem_name']
    db=pd.read_csv(f'/home/data_folder/osm_data/road_gno_cat_raw/{grid_name}.csv')
    db1=pd.pivot_table(db,index=db.gno,values=['fclass_length_km'],columns=['road_category'])
    db1.columns=db1.columns.droplevel()
    db2=db1.reset_index()
    #ed=rd4.groupby(['Maille','Maille_X','Maille_Y'])['length_km'].agg('sum')
    km_db=gp.read_file(f'/home/data_folder/osm_data/5x5_1km_grids_ea/{grid_name}.shp')
    km_db1=km_db[['gn','geometry']]
    km_db1.columns=['gno','geometry']
    db2=pd.merge(km_db1,db2,on='gno',how='left')
    db2["lon"] = db2['geometry'].centroid.x
    db2["lat"] = db2['geometry'].centroid.y
    #db3=db2[['gno','lat','lon','Nomotorway','primary','secondary','tertiary','unclassified']]
    db2 = db2.drop('geometry', axis=1)
    db2.to_csv(f'/home/data_folder/osm_data/road_gno_cat_raw/road_class_{grid_name}.csv',index=False)
    print("--- %s seconds ---" % (time.time() - start_time))   



--- 28.22124147415161 seconds ---
--- 57.14632964134216 seconds ---
--- 85.14412117004395 seconds ---
--- 113.17185163497925 seconds ---
--- 139.89145374298096 seconds ---
--- 167.90957188606262 seconds ---
--- 200.67765760421753 seconds ---
--- 233.08927059173584 seconds ---
--- 264.9049639701843 seconds ---
--- 299.18258333206177 seconds ---
--- 335.87677931785583 seconds ---
--- 373.7629692554474 seconds ---
--- 407.06586503982544 seconds ---
--- 442.652015209198 seconds ---
--- 476.01923084259033 seconds ---
--- 599.4273293018341 seconds ---
--- 631.9030044078827 seconds ---
--- 666.395359992981 seconds ---
--- 698.5828421115875 seconds ---
--- 731.4879989624023 seconds ---
--- 765.6095554828644 seconds ---
--- 796.7477140426636 seconds ---
--- 831.0619306564331 seconds ---
--- 864.3421015739441 seconds ---
--- 898.076672077179 seconds ---
--- 933.7542662620544 seconds ---
--- 967.6346912384033 seconds ---
--- 1007.333190202713 seconds ---
--- 1047.381638765335 seconds ---
--- 1083

In [33]:
db2

,gno,geometry,Nomotorway,tertiary,unclassified,lon,lat
0,1,"POLYGON ((19.99958 9.99958, 20.00958 9.99958, ...",NaN,NaN,NaN,20.004583,9.994583
1,2,"POLYGON ((19.99958 9.98958, 20.00958 9.98958, ...",NaN,NaN,NaN,20.004583,9.984583
2,3,"POLYGON ((19.99958 9.97958, 20.00958 9.97958, ...",NaN,NaN,NaN,20.004583,9.974583
3,4,"POLYGON ((19.99958 9.96958, 20.00958 9.96958, ...",NaN,NaN,NaN,20.004583,9.964583
4,5,"POLYGON ((19.99958 9.95958, 20.00958 9.95958, ...",NaN,NaN,NaN,20.004583,9.954583
...,...,...,...,...,...,...,...
249995,249996,"POLYGON ((24.98958 5.04958, 24.99958 5.04958, ...",NaN,NaN,NaN,24.994583,5.044583
249996,249997,"POLYGON ((24.98958 5.03958, 24.99958 5.03958, ...",NaN,NaN,NaN,24.994583,5.034583
249997,249998,"POLYGON ((24.98958 5.02958, 24.99958 5.02958, ...",0.213943,NaN,NaN,24.994583,5.024583
249998,249999,"POLYGON ((24.98958 5.01958, 24.99958 5.01958, ...",NaN,NaN,NaN,24.994583,5.014583


In [23]:
#db2=db1.reset_index()
db1.columns=db1.columns.droplevel()
db2=db1.reset_index()
db2

road_category,gno,Nomotorway,primary,secondary,tertiary,unclassified
0,472,NaN,NaN,NaN,NaN,0.327574
1,473,0.076447,NaN,NaN,NaN,0.881863
2,474,0.243894,NaN,NaN,NaN,NaN
3,475,0.209879,NaN,NaN,NaN,0.595777
4,476,0.440477,NaN,NaN,NaN,0.367973
...,...,...,...,...,...,...
3386,249972,0.056578,NaN,NaN,0.249558,NaN
3387,249973,0.159273,1.233041,NaN,0.285443,NaN
3388,249974,NaN,0.073772,NaN,NaN,NaN
3389,249989,0.609612,NaN,NaN,NaN,NaN


### get all the road types in ea and make dictionary

In [3]:
import gc
import geopandas as gp
import pandas as pd

grid_db=gp.read_file('/home/data_folder/osm_data/ea_5x5_grid.shp')


d_cont=[]
for idx, row in grid_db.iterrows():
    grid_name=row['dem_name']
    output_shp=f'/home/data_folder/osm_data/{grid_name}_road_grid_stat.shp'
    db=gp.read_file(output_shp)
    db1=db.drop_duplicates('fclass')
    d_cont.append(db1)
    print(row['dem_name'])
    del [[db,db1]]
    gc.collect()
    db=pd.DataFrame()
    db1=pd.DataFrame()
    

fc_db=pd.concat(d_cont)

n00e025_dem
n00e030_dem
n00e035_dem
n00e040_dem
n00e045_dem
n05e020_dem
n05e025_dem
n05e030_dem
n05e035_dem
n05e040_dem
n05e045_dem
n05e050_dem
n10e020_dem
n10e025_dem
n10e030_dem
n10e035_dem
n10e040_dem
n10e045_dem
n10e050_dem
n15e020_dem
n15e025_dem
n15e030_dem
n15e035_dem
n20e025_dem
n20e030_dem
n20e035_dem
s05e025_dem
s05e030_dem
s05e035_dem
s05e040_dem
s10e025_dem
s10e030_dem
s10e035_dem
s15e030_dem
s15e035_dem
s15e040_dem


In [7]:
fc_db=fc_db.drop_duplicates('fclass')
fc_db1=fc_db[['fclass','gno']]

In [8]:
dict_db=pd.read_csv('/home/ea-ibf-climada/exposure/roads/road_dict.csv')
dict_db.columns=['fclass','cat2']

_=pd.merge(fc_db1,dict_db,on='fclass')
_

,fclass,gno,cat2
0,primary,249467,primary
1,secondary,245447,secondary
2,tertiary,226409,tertiary
3,track,235273,Nomotorway
4,unclassified,218980,unclassified
5,residential,227340,tertiary
6,service,228843,tertiary
7,path,229337,Nomotorway
8,living_street,228840,tertiary
9,footway,228840,Nomotorway


In [ ]:
outSHP=f'{grid}_roadDensity.shp'
    rd2=gp.read_file(outSHP)
    stli=rd2.fclass.tolist()
    stli2=[]
    for lt in stli:
        L2 = str(lt)
        L3=[L2]
        stli2.append(L3)
    a=[]
    for lt in stli2:
        L2 = [bus[x] for x in lt]
        a.append(L2)
    ab=np.array(a)
    rd2['RoadType']=ab
    roadcats=(rd2.drop_duplicates('RoadType'))['RoadType'].tolist()
    for roadcat in roadcats:
        rd3=rd2[rd2['RoadType']==roadcat]
        rd4=gp.GeoDataFrame(rd3)
        rd4.to_file(driver = 'ESRI Shapefile', filename=f'{grid}_roadcat.shp')